In [1]:
import numpy as np
import math
# %timeit -n10 

### Fastest nth root

In [2]:
x = np.random.random((500, 500))
y = np.random.random((500, 500))

In [3]:
%timeit -n20 x**(1/40)

3.94 ms ± 55.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%timeit -n20 np.power(x, 1/40)

3.99 ms ± 42.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastest random float [0, 1[

In [5]:
from random import random as fastRandom
%timeit -n400 fastRandom()

45.7 ns ± 5.89 ns per loop (mean ± std. dev. of 7 runs, 40000 loops each)


In [6]:
%timeit -n400 np.random.random()

355 ns ± 34 ns per loop (mean ± std. dev. of 7 runs, 40000 loops each)


### Fastest power

In [7]:
%timeit -n20 x ** y
%timeit -n20 np.power(x, y)

4.26 ms ± 69.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.16 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit -n20 x ** 2
%timeit -n20 np.power(x, 2)

92.9 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.18 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### ==> Numpy.power is faster for small powers
#### ==> Python ** is faster for large powers

### Fastest Sqrt

In [9]:
%timeit -n50 x ** (1/2)
%timeit -n50 np.sqrt(x)

192 µs ± 7.69 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)
195 µs ± 4.85 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


### Fastest Descending sorting in numpy

In [10]:
a = np.random.rand(40,40,80)

In [11]:
%timeit -n10 np.flip(np.sort(a, axis=-1, kind='quicksort'), axis=-1)
%timeit -n10 np.sort(a, axis=-1, kind='quicksort')[::-1]
%timeit -n10 (-np.sort(-a, axis=-1, kind='quicksort'))

2.68 ms ± 54.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.66 ms ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.78 ms ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Fastest derangment
i.e. random shuffling such that no element is at the same place as in the begining, i.e. without fixed point

In [12]:
def random_derangement_1(n):
    while True:
        v = np.arange(n)
        for j in np.arange(n - 1, -1, -1):
            p = np.random.randint(0, j+1)
            if v[p] == j:
                break
            else:
                v[j], v[p] = v[p], v[j]
        else:
            if v[0] != 0:
                return v

In [13]:
def random_derangement_2(N):
    original = np.arange(N)
    new = np.random.permutation(N)
    same = np.where(original == new)[0]
    while len(same) != 0:
        swap = same[np.random.permutation(len(same))]
        new[same] = new[swap]
        same = np.where(original == new)[0]
        if len(same) == 1:
            swap = np.random.randint(0, N)
            new[[same[0], swap]] = new[[swap, same[0]]]
    return new

In [14]:
import itertools

In [15]:
def random_derangement_3(n):
    list_ex = np.arange(n)
    i = filter(lambda p: not any(i1==i2 for i1,i2 in zip(list_ex, p)), itertools.permutations(list_ex, len(list_ex)))
    return next(i)

In [16]:
%timeit -n10 random_derangement_1(2000)

6.03 ms ± 230 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit -n90 random_derangement_2(2000)

68.1 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 900 loops each)


In [18]:
# Wont run the test...
#%timeit -n100 random_derangement_3(2000)

In [19]:
%timeit -n30 np.random.permutation(2000)

41.8 µs ± 6.23 µs per loop (mean ± std. dev. of 7 runs, 300 loops each)


#### Result
random_derangement_2 is the clear winner

## Fastest random in multiple distribs

In [20]:
import scipy.stats as stats
# Middle | Ignores half for crossing over
middle = lambda n : int(n/2)
# Uniform | Okay but in nature crossing over isn't uniform
uniform = lambda n : np.random.randint(n)
# Uniform Middle | Better, but still ignores the extremis
uniformMiddle = lambda n: np.random.randint(int(n/2 - n/4), int(n/2 + n/4))
# Normal | Better, but small proba that x >= n or x < 0
normal = lambda n : int(np.random.normal(n*0.5, n*0.33))
# Normal truncated | Best solution, fits nature, but rather costly
normalTrucated = lambda n : int(stats.truncnorm(-1.5, 3-((3+n)/n), loc=n/2, scale=n*0.3).rvs(1)[0])
# Normal truncated | Best solution, fits nature, but rather costly
def normalTrucatedSingle(n):
    nrml = np.random.normal(n*0.5, n*0.33)
    if nrml < 0:
        return 0
    elif nrml >= n:
        return n-1
    return nrml
def normalTrucatedMultiple(n, size=1):
    return np.random.normal(n*0.5, n*0.33, size=size).astype(int).clip(0, n-1)

In [21]:
%timeit -n30 middle(60000)

212 ns ± 18 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [22]:
%timeit -n30 uniform(60000)

1.15 µs ± 75.9 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [23]:
%timeit -n30 uniformMiddle(60000)

1.65 µs ± 77.4 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [24]:
%timeit -n30 normal(60000)

1.93 µs ± 746 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [25]:
%timeit -n3 normalTrucated(600)

647 µs ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 300 loops each)


In [26]:
%timeit -n30 normalTrucatedSingle(60000)

2.03 µs ± 918 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [27]:
%timeit -n30 normalTrucatedMultiple(60000, 1)

3.89 µs ± 160 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [28]:
%timeit -n30 normalTrucatedMultiple(60000, 500)

17.4 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [29]:
%timeit -n30 normalTrucatedMultiple(60000, 1500)

43.2 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [30]:
%timeit -n30 normalTrucatedMultiple(60000, 5000)

134 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [31]:
t = np.array(1.0)
%timeit -n30 t.astype(int)
%timeit -n30 int(t)

296 ns ± 63.5 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)
134 ns ± 4.02 ns per loop (mean ± std. dev. of 7 runs, 3000 loops each)


In [32]:
%timeit -n30 123456789%2==1

24.2 ns ± 2.49 ns per loop (mean ± std. dev. of 7 runs, 300000 loops each)


In [33]:
%timeit -n30 123456789&1==1

24.3 ns ± 1.95 ns per loop (mean ± std. dev. of 7 runs, 300000 loops each)


In [34]:
from scipy.optimize import rosen as rosenSciPy
rosenArray = lambda x : sum(100.0*(x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)
rosenArray(np.array([5,7,4,9]))

239861.0

In [35]:
rosenBivariate = lambda x, y: (1-x) ** 2 + 100*(y-x**2)**2
rosenBivariate(5,7)

32416

In [36]:
%timeit -n10 rosenArray(np.array([1433,4521, 3432, 4324] * 90))

59.1 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [37]:
L = [1433,4521,3432,4324] * 8
%timeit -n30 sum([rosenBivariate(L[i], L[i+1]) for i in range(len(L)-1)])

34.9 µs ± 644 ns per loop (mean ± std. dev. of 7 runs, 30000 loops each)


In [38]:
%timeit -n10 rosenSciPy(np.array([1433,4521, 3432, 4324] * 90))

42.8 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [39]:
rosenBivariate(0.32, 0.879)

60.773156

In [40]:
a = np.random.rand(10)

In [41]:
size = 3000
a = np.random.random_sample((size,size))
b = np.random.random_sample((size,size))
%timeit np.dot(a,b)

394 ms ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
matricesElementCount = lambda psi : [psi[i - 1] * psi[i] for i in range(1, len(psi))]

In [58]:
psi = np.arange(1000)

In [61]:
%timeit -n200 matricesElementCount(psi)

233 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 200 loops each)


In [66]:
%timeit -n200 np.multiply(psi[:-1], psi[1:])

2.23 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 200 loops each)
